# Lecture 6 - Networks

In [1]:
# # Dependencies: Uncomment and run this cell if you don't have these packages 
# using Pkg
# Pkg.add("JuMP")
# Pkg.add("Gurobi") # You need Gurobi for the lazy constraints
# Pkg.add("StatsBase")
# Pkg.add("Graphs")
# Pkg.add("GraphIO")

In [2]:
using Graphs, GraphIO
using JuMP, Gurobi
using Random, StatsBase

## Load graph input

We'll use a graph from the [Stanford Large Network Dataset Collection](https://snap.stanford.edu/data/).

In [3]:
@time g = squash(loadgraph(
	"data/amazon.txt", 
	GraphIO.EdgeList.EdgeListFormat()));
println(length(vertices(g)), " vertices, ", g.ne, " edges")

  1.013948 seconds (11.48 M allocations: 626.225 MiB, 15.28% gc time)
262111 vertices, 1234877 edges


## Specialized Algorithm (Dijkstra)

First, we'll run Dijkstra's algorithm. This code will give us the shortest paths from node 1 to all other nodes.

In [4]:
s = 1
@time ds = Graphs.dijkstra_shortest_paths(g, [s]);

  0.168979 seconds (292.10 k allocations: 32.053 MiB, 49.91% compilation time)


## General Solver

Now, we'll solve the shortest path problem using LP. First, let's load the Gurobi optimizer. 

In [5]:
env = Gurobi.Env()
optimizer = optimizer_with_attributes(() -> Gurobi.Optimizer(env), "OutputFlag" => 0);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-18


Here's a function that builds an LP to solve the shortest path problem from source s to sink t. (Note that with Dijkstra's algorithm, we could have multiple sources and sinks). 

In [6]:
function shortest_path_lp(g::SimpleDiGraph, s::Int, t::Int)

    # Extract graph parameters
    edge_list = [(e.src, e.dst) for e in Graphs.edges(g)]
    V = length(vertices(g))
    
    # RHS of the flow balance constraint
    rhs(u::Int, s::Int, t::Int) = (u == s) ? 1 : (u == t) ? -1 : 0

    # Build model
    model = JuMP.Model(optimizer)
    JuMP.set_silent(model)
	@variable(model, 0 <= x[edge_list] <= 1);
	@constraint(model, 
		[u = 1:V],
		sum(x[(u, v)] for v in g.fadjlist[u]) - 
		sum(x[(v, u)] for v in g.badjlist[u]) == rhs(u, s, t));
	@objective(model, Min, sum(x[(u, v)] for (u, v) in edge_list))
    optimize!(model)
    
    return objective_value(model), solve_time(model)

end

shortest_path_lp (generic function with 1 method)

Now we'll solve a few instances of the shortest path LP and see how it does.

In [7]:
Random.seed!(1)
solve_times = Float64[]
for it in 1:5
    t = rand(1:length(vertices(g)))
    dist_t, time_t = shortest_path_lp(g, s, t)
    @assert isapprox(ds.dists[t], dist_t) # Check that our model matches Dijkstra's algorithm
    push!(solve_times, time_t)
end
println("Average solve time: ", mean(solve_times), " seconds")

Average solve time: 3.85159854888916 seconds
